In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
# from snorkel.parser import TextDocPreprocessor

# doc_preprocessor = TextDocPreprocessor('disasterData/doc1.txt')

In [3]:
from snorkel.parser import TextDocPreprocessor2
doc_preprocessor = TextDocPreprocessor2('data2')

In [4]:
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser


n_docs=5

corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=n_docs)

Clearing existing...
Running UDF...
[========                                ] 20%data2/doc7
[========                                ] 20%data2/doc40
[================                        ] 40%data2/doc116
[========================                ] 60%data2/doc137
[================================        ] 80%data2/doc19
[========================================] 100%data2/doc20
[================================================] 120%data2/doc71
data2/doc14
data2/doc95
data2/doc59
data2/doc58
data2/doc3
data2/doc141
data2/doc56
data2/doc104
data2/doc13
data2/doc72
data2/doc129
data2/doc9
data2/doc78
data2/doc106
data2/doc93
data2/doc131
data2/doc51
data2/doc8
data2/doc94
data2/doc47
data2/doc23
data2/doc144
data2/doc67
data2/doc74
data2/doc44
data2/doc70
data2/doc64
data2/doc133
data2/doc124
data2/doc118
data2/doc111
data2/doc100
data2/doc82
data2/doc114
data2/doc123
data2/doc69
data2/doc4
data2/doc103
data2/doc12
data2/doc80
data2/doc149
data2/doc21
data2/doc61
data2/doc79
data2/do

In [5]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())


('Documents:', 151)
('Sentences:', 1609)


In [6]:
from snorkel.models import candidate_subclass

Disaster = candidate_subclass('Disaster', ['Name','Location'])

In [7]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import DateMatcher, LocationMatcher
ngrams         = Ngrams(n_max=7)
#person_matcher = PersonMatcher(longest_match_only=True)
location_matcher = LocationMatcher(longest_match_only=True)
date_matcher=DateMatcher(longest_match_only=True)
#dict=['earthquake','flood','cyclone','fire']
#dictionary_matcher=DictionaryMatch(d=dict)
cand_extractor = CandidateExtractor(Disaster, [ngrams, ngrams], [date_matcher, location_matcher])


In [9]:
from snorkel.models import Document
#from util import number_of_people


docs = session.query(Document)

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 3 == 2:
            dev_sents.add(s)
        elif i % 3 == 1:
            dev_sents.add(s)
        else:
            dev_sents.add(s)


In [10]:
print(len(dev_sents),len(train_sents),len(test_sents))

(1609, 0, 0)


In [11]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(Disaster).filter(Disaster.split == i).count())



Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 0)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 89)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 0)
CPU times: user 12.5 s, sys: 432 ms, total: 12.9 s
Wall time: 13 s


In [11]:
from lxml import etree as ET
import csv
import sys
reload(sys)
sys.setdefaultencoding('utf8')
print(1)


#tree = ET.parse("../../rnd/DisasterAnnotatedDocs-English-AUKBC/doc1_tagged.xml")
#tree = ET.parse("../../rnd/gold")
#root = tree.getroot()
'''i=1
def input_file():
    global i
    tree = ET.parse("../../rnd/gold/doc"+str(i)+"_tagged.xml")
    i=i+1
    print(tree)
    return(tree)'''    
    
#parent_map = dict((c, p) for p in tree.getiterator() for c in p)
with open('gold_labels.tsv', 'w') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        writer.writerow(["Name", "Location", "label"])
        for i in range(1,152):
            if i!=31:
                    tree = ET.parse("../../rnd/gold/doc"+str(i)+"_tagged.xml")
                    root = tree.getroot()
                    parent_map = dict((c, p) for p in tree.getiterator() for c in p)
                    print("doc"+str(i))
            
        #tree=input_file()

                    for nateve in root.findall(".//NATURAL_EVENT"):
    
                            a=0
                            b=0
                            s=''
                            r=''
                            print nateve.attrib["ID"],ET.tostring(nateve)
                            place = ""
                            time = ""
                            eventid = ""
                            links = root.findall(".//LINK[@EVENT_ARG=\"{}\"]".format(nateve.attrib["ID"]))
                            for link in links:

                                x = parent_map[link]
                                if(x.tag=="PLACE-ARG"):
                                    print x.tag
                                    for c in x:
                                        if(c.tag == "W"):
                                            a=1
                                            print c.text
                                            s=c.text+s
                            links = root.findall(".//LINK[@EVENT_ARG=\"{}\"]".format(nateve.attrib["ID"]))
                            for link in links:

                                x = parent_map[link]
                                if(x.tag=="TIME-ARG"):
                                    print x.tag
                                    for c in x:
                                        if(c.tag == "W"):
                                            b=1
                                            print c.text
                                            r=c.text+r
                                            
                            if(a==1 and b==1):
                                    writer.writerow([r,s,str(1)])
                            elif(a==1 and b==0):
                 #record_file.write(s+"  "+str("")+"  "+str(-1)+"\n")
                                       writer.writerow(["",s,str(-1)])
                            elif(a==0 and b==1):
                 #record_file.write(str("")+"  "+r+"  "+str(-1)+"\n")
                                        writer.writerow([r,"",str(-1)+"\n"])
                            else:
                 #record_file.write(str("")+"  "+str("")+"  "+str(-1)+"\n")'''
                                       writer.writerow(["","",str(-1)])

                    
                    
                    for nateve in root.findall(".//MAN_MADE_EVENT"):
    
                            a=0
                            b=0
                            s=''
                            r=''
                            print nateve.attrib["ID"],ET.tostring(nateve)
                            place = ""
                            time = ""
                            eventid = ""
                            links = root.findall(".//LINK[@EVENT_ARG=\"{}\"]".format(nateve.attrib["ID"]))
                            for link in links:

                                x = parent_map[link]
                                if(x.tag=="PLACE-ARG"):
                                    print x.tag
                                    for c in x:
                                        if(c.tag == "W"):
                                            a=1
                                            print c.text
                                            s=c.text+s
                            links = root.findall(".//LINK[@EVENT_ARG=\"{}\"]".format(nateve.attrib["ID"]))
                            for link in links:

                                x = parent_map[link]
                                if(x.tag=="TIME-ARG"):
                                    print x.tag
                                    for c in x:
                                        if(c.tag == "W"):
                                            b=1
                                            print c.text
                                            r=c.text+r


                    #print(type(c.text))
    
                            if(a==1 and b==1):
                                    writer.writerow([r,s,str(1)])
                            elif(a==1 and b==0):
                 #record_file.write(r+"  "+str("")+"  "+str(-1)+"\n")
                                       writer.writerow(["",s,str(-1)])
                            elif(a==0 and b==1):
                 #record_file.write(str("")+"  "+s+"  "+str(-1)+"\n")
                                        writer.writerow([r,"",str(-1)+"\n"])
                            else:
                 #record_file.write(str("")+"  "+str("")+"  "+str(-1)+"\n")'''
                                       writer.writerow(["","",str(-1)])


    
#     print(place.tag,place.text)
#     for x in nateve:
#         print x.tag,x.text
# root.find(".//entity[@id=\"{}\"]".format(cand['entity1id']))

In [15]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

from util import load_external_labels

%time missed = load_external_labels(session, Disaster, annotator_name='gold')

SyntaxError: invalid syntax (util.py, line 33)

In [16]:
#for c in session.query(Disaster).filter(Disaster.split == 0).all():
    #print(c)

In [2]:
labeled = []
for c in session.query(Disaster).filter(Disaster.split == 0).all():
    if LF_mdisaster(c) != 0:
        labeled.append(c)
print("Number labeled:", len(labeled))

NameError: name 'session' is not defined

In [38]:
labeled = []
for c in session.query(Disaster).filter(Disaster.split == 1).all():
    if LF_mandisaster(c) != 0:
        labeled.append(c)
print("Number labeled:", len(labeled))

In [47]:
from snorkel.viewer import SentenceNgramViewer

SentenceNgramViewer(labeled, session)

<IPython.core.display.Javascript object>

A Jupyter Widget

In [21]:
print('hi')

In [12]:
import re
from snorkel.lf_helpers import (
    get_between_tokens,
    get_text_between, get_tagged_text,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

                   
meteorological_disasters = { 'storm', 'bizzard', 'tornado', 'hurricane', 'cyclone', 'avalanches', 'heatwave','coldwave' 'drought', 'hailstorm'}
hydrological_disasters = { 'flood' , 'tsunami', 'limnic eruption' , 'famine' , 'forest fire' }

geological_disasters = { 'earthquake', 'volcano' , 'rock fall'}

                   
biological_disasters = { 'epidemic', 'pandemic' , 'influenza'}

manmade_disasters = { 'blast', 'bomb explosion', 'bomb blast', 'bomb', 'bombings', 'surgical strike', 'shooting', 'firing', 'gun fire' , 'riots' , 'armed conflict'}

def LF_mdisaster(c):
     return 1 if len(meteorological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_hdisaster(c):
    return 1 if len(hydrological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_gdisaster(c):
    return 1 if len(geological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_bdisaster(c):
    return 1 if len(biological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_mandisaster(c):
    return 1 if len(manmade_disasters.intersection(get_between_tokens(c))) > 0 else 0
def LF_tdisaster(c):
      return 1 if 'train’' in get_between_tokens(c) and 'collided’' in get_right_tokens(c) else 0
def LF_killed(c):
    return 1 if re.search (r'{{A}}+\w+\s+killed+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I)  else 0

def LF_injured(c):
    return 1 if re.search (r'{{A}}+\w+\s+injured+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I)  else 0

def LF_dead(c):
    return 1 if re.search (r'{{A}}+\w+\s+dead+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_damage(c):
    return 1 if re.search (r'{{A}}+\w+\s+heavy+\s+damage+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_conflict(c):
    return 1 if re.search (r'{{A}}+\w+\s+conflict+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0


def ltp(x):
    return '(' + '|'.join(x) + ')'

trigger1 = ['killed', 'kills', 'killing']

def LF_kill(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger1) + '.\w', 1)
trigger2 = ['violence', 'violent', 'rioting']
def LF_violence(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger2) + '.\w', 1)
trigger3 = ['crashed’,’collided’,’hit’']

def LF_crash(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger3) + '.\w', 1)

trigger4 = ['surgical strike', 'strike', 'struck', 'hit the target']

def LF_strike(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger4) + '.\w', 1)

trigger5 = ['cold', 'heat']

def LF_wave(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger4) +"wave"+ '.\w', 1)
def LF_cold(c):
    return 1 if re.search (r'{{A}}+\w+\s+cold+arctic +air+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

trigger6 = ['flood' ,'floods' , 'flooding', 'flooded']
def LF_flood(c):
     return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger4) + '.\w', 1)

trigger7 = ['eruption' ,'erupting' , 'erupted',  'erupt']

def LF_eruption(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger7) + '.\w', 1)

trigger8 = ['fell off' ,'slide', 'fall']

#def LF_rock(c):
    #return rule_regex_search_btw_AB(c, '.\w' + rock + ltp(trigger8) + '.\w', 1)

trigger9 = ['cholera' ,'ebola', 'dengue']

def LF_epidemic(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger9) + '.\w', 1)


def LF_tsunami(c):
    return 1 if re.search (r'{{A}}+\w+\s+tsunami+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_confront(c):
    return 1 if re.search (r'{{A}}+\w+\s+armed+confrontation+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_rammed(c):
    return 1 if re.search (r'{{A}}+LF_rammed\w+\s+rammed+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_smashed(c):
    return 1 if re.search (r'{{A}}+\w+\s+smashed+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_exploded(c):
    return 1 if re.search (r'{{A}}+\w+\s+exploded+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_shipwrecked(c):
    return 1 if re.search (r'{{A}}+\w+\s+shipwrecked+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_nuclearmeltdown(c):
    return 1 if re.search (r'{{A}}+\w+\s+nuclear meltdown.+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_tornado(c):
    return 1 if re.search (r'{{A}}+\w+\s+tornado(es)+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_limniceruption(c):
    return 1 if re.search (r'{{A}}+\w+\s+limnic eruption+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_wildfire(c):
    return 1 if re.search (r'{{A}}+\w+\s+wildfire(es)+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

In [1]:
import snorkel.annotations 

In [13]:
LFs = [
     LF_mdisaster,  LF_gdisaster,  LF_hdisaster,  LF_bdisaster,  LF_mandisaster, LF_tdisaster, LF_wildfire,LF_limniceruption, LF_tornado
,LF_nuclearmeltdown, LF_shipwrecked  ,LF_exploded, LF_smashed, LF_rammed,  LF_confront,LF_tsunami ,LF_epidemic,LF_eruption,LF_flood,
LF_wave,LF_strike, LF_crash, LF_violence, LF_kill, LF_conflict, LF_damage, LF_dead, LF_injured, LF_kill ]
 



In [14]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)

In [44]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

np.random.seed(1701)
%time L_train = labeler.apply(split=1)
L_train

In [ ]:
print(L_train)

In [49]:
%time L_train = labeler.load_matrix(session, split=0)
L_train



CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.13 ms


<0x0 sparse matrix of type '<type 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [18]:
L_train.lf_stats(session)

,j,Coverage,Overlaps,Conflicts
LF_mdisaster,0,0.067416,0.000000,0.0
LF_gdisaster,1,0.101124,0.033708,0.0
LF_hdisaster,2,0.056180,0.033708,0.0
LF_bdisaster,3,0.000000,0.000000,0.0
LF_mandisaster,4,0.000000,0.000000,0.0
LF_tdisaster,5,0.000000,0.000000,0.0
LF_wildfire,6,0.000000,0.000000,0.0
LF_limniceruption,7,0.000000,0.000000,0.0
LF_tornado,8,0.000000,0.000000,0.0
LF_nuclearmeltdown,9,0.000000,0.000000,0.0


In [45]:
L_train.get_candidate(session, 0)

KeyError: 0

In [17]:
L_train.get_key(session, 0)

LabelKey (LF_mdisaster)

In [19]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()
gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)



In [20]:
gen_model.weights.lf_accuracy


array([ 0.1285002 ,  0.12176998,  0.11403378,  0.07474443,  0.09075024,
        0.0767384 ,  0.07119023,  0.06212787,  0.06721162,  0.05644994,
        0.08115259,  0.04437729,  0.07741764,  0.07553978,  0.05798367,
        0.07684375,  0.04994734,  0.08945969,  0.06329529,  0.07153999,
        0.07115216,  0.07100951,  0.06979111,  0.07470675,  0.09042339,
        0.04918879,  0.07346887,  0.08018315])

In [19]:
gen_model.learned_lf_stats()

,Accuracy,Coverage,Precision,Recall
0,0.566727,0.6684,0.557744,0.377881
1,0.567244,0.6759,0.557480,0.377477
2,0.558161,0.6654,0.550863,0.374444
3,0.533927,0.6735,0.527594,0.355641
4,0.543475,0.6705,0.543844,0.372422
5,0.542526,0.6737,0.535671,0.358269
6,0.548233,0.6707,0.539870,0.369592
7,0.529603,0.6773,0.524576,0.356045
8,0.537040,0.6709,0.532004,0.361302
9,0.531725,0.6714,0.532051,0.369187


In [21]:
train_marginals = gen_model.marginals(L_train)

In [22]:
def predict_labels(marginals):
    predicted_labels=[]
    for i in marginals:
        if(i<0.5): 
            predicted_labels.append(-1)
        else:
            predicted_labels.append(1)
    return predicted_labels
x=predict_labels(train_marginals)
print(type(x))

print(x)

In [35]:
import csv
import os
import numpy as np
my_list=[]
with open("/home/shreshtha/2nd_sem/seminar/snorkel-master/gold_labels.tsv") as f:
    reader = csv.reader(f)
    my_list = [row[-1] for row in reader]
list=np.array(my_list)
list.append(1)
print(list)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [36]:
import csv
import os
import numpy as np
my_list=[]
with open("/home/shreshtha/2nd_sem/seminar/snorkel-master/gold_labels.tsv") as f:
    next(f)
    reader = csv.reader(f,delimiter='\t')
    
    my_list = [row[-1] for row in reader]

for i in range(0,7):    
    my_list.append('-1')
print(len(my_list))
list=np.array(my_list)
print(list)
#print(size(list))
print('\n')

In [37]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
x=np.array(x)
precision_recall_fscore_support(x, list, average='macro')

ValueError: Found input variables with inconsistent numbers of samples: [89, 779]

In [23]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

In [24]:
L_dev = labeler.apply(split=1)


In [25]:
tp, fp, tn, fn = gen_model.error_analysis(session, L_dev, L_gold_dev)


In [26]:
L_dev.lf_stats(session, L_gold_dev, gen_model.learned_lf_stats()['Accuracy'])


ValueError: sequence too large; cannot be greater than 32

In [3]:
print("hi")

hi


In [1]:
from snorkel.lf_helpers import test_LF
tp, fp, tn, fn = test_LF(session, LF_mdisaster, split=1, annotator_name='gold')



NameError: name 'session' is not defined